 # Best Artworks of All Time
 ## Collection of Paintings of the 50 Most Influential Artists of All Time
 ### https://www.kaggle.com/ikarus777/best-artworks-of-all-time

Я собираюсь решить проблему распознавания жанра по картине. Поскольку эта задача носит образовательный характер, я могу позволить себе некоторые методы, связанные с формированием нового набора данных и предварительной обработкой данных, чтобы облегчить себе жизнь.

In [2]:
import os
import cv2 as cv
import numpy as np
import pandas as pd



Первые пять записей из датасета.

In [3]:
artists = pd.read_csv('artists.csv')
artists.head()



,id,name,years,genre,nationality,bio,wikipedia,paintings
0,0,Amedeo Modigliani,1884 - 1920,Expressionism,Italian,Amedeo Clemente Modigliani (Italian pronunciat...,http://en.wikipedia.org/wiki/Amedeo_Modigliani,193
1,1,Vasiliy Kandinskiy,1866 - 1944,"Expressionism,Abstractionism",Russian,Wassily Wassilyevich Kandinsky (Russian: Васи́...,http://en.wikipedia.org/wiki/Wassily_Kandinsky,88
2,2,Diego Rivera,1886 - 1957,"Social Realism,Muralism",Mexican,Diego María de la Concepción Juan Nepomuceno E...,http://en.wikipedia.org/wiki/Diego_Rivera,70
3,3,Claude Monet,1840 - 1926,Impressionism,French,Oscar-Claude Monet (; French: [klod mɔnɛ]; 14 ...,http://en.wikipedia.org/wiki/Claude_Monet,73
4,4,Rene Magritte,1898 - 1967,"Surrealism,Impressionism",Belgian,René François Ghislain Magritte (French: [ʁəne...,http://en.wikipedia.org/wiki/René_Magritte,194


In [4]:
artists = artists.drop(['bio', 'wikipedia', 'years'], 1)
artists = artists.sort_values(by=['paintings'])
artists = artists.reset_index(drop=True)



In [5]:
for i, row in artists.iterrows():
    artists.at[i, 'name'] = artists.at[i, 'name'].replace(" ", "_")
    artists.at[i, 'genre'] = artists.at[i, 'genre'].replace(" ", "_")
(artists.head())



,id,name,genre,nationality,paintings
0,49,Jackson_Pollock,Abstract_Expressionism,American,24
1,39,Eugene_Delacroix,Romanticism,French,31
2,41,Georges_Seurat,Post-Impressionism,French,43
3,38,Paul_Cezanne,Post-Impressionism,French,47
4,48,Michelangelo,High_Renaissance,Italian,49


In [7]:
artists_by_genre = artists.drop(['nationality'], 1)
artists_by_genre = artists_by_genre.drop_duplicates(
    subset='genre', keep='last')
for i, row in artists_by_genre.iterrows():
    genres = artists_by_genre.at[i, 'genre'].split(',')
    artists_by_genre = artists_by_genre.drop(i)
    for genre in genres:
        # find genre in "genre" column
        if (artists_by_genre['genre'][artists_by_genre['genre'] == genre].empty):
            row.at['genre'] = genre
            artists_by_genre = artists_by_genre.append(row)
artists_by_genre = artists_by_genre.sort_values(by=['paintings']).drop_duplicates(
    subset=['genre'], keep='last').drop_duplicates(subset=['name'], keep='last')
artists_by_genre = artists_by_genre[artists_by_genre.paintings >= 100].reset_index(
    drop=True)
artists_by_genre


,id,name,genre,paintings
0,9,Gustav_Klimt,Art_Nouveau,117
1,23,Giotto_di_Bondone,Proto_Renaissance,119
2,11,Kazimir_Malevich,Suprematism,126
3,5,Salvador_Dali,Surrealism,139
4,26,Leonardo_da_Vinci,High_Renaissance,143
5,24,Sandro_Botticelli,Early_Renaissance,164
6,12,Mikhail_Vrubel,Symbolism,171
7,45,Andy_Warhol,Pop_Art,181
8,0,Amedeo_Modigliani,Expressionism,193
9,22,Marc_Chagall,Primitivism,239


У нас есть набор данных из 16 классов. Из классов мы выберем 100 изображений для обучения, а из оставшихся 10 - для проверки.

In [8]:
artists = artists_by_genre["name"].to_list()
for i, artist in enumerate(artists):
    if artist == 'Albrecht_Dürer':
        artists[i] = 'Albrecht_Dürer'
genres = artists_by_genre["genre"].to_list()
name_genre = dict(zip(artists, genres))


In [10]:
groot_data_path = "images"
for path, subdirs, files in os.walk(groot_data_path):
    i = 1
    for artist in artists:
        if artist in path and 'resized' not in path:
            for name in files:
                if (i > 110):
                    break
                src_im = cv.imread(os.path.join(
                    path, name), cv.IMREAD_GRAYSCALE)
                if src_im is None:
                    print("bad read:", os.path.join(
                        path, name))
                height, width = src_im.shape
                dst_im = src_im
                if height < width:
                    dst_im = src_im[:, width//2 -
                                    height//2:width//2 + height//2]
                elif height > width:
                    dst_im = src_im[height//2 - width //
                                    2:height//2 + width//2, :]

                dst_im = cv.resize(
                    dst_im, (64, 64), interpolation=cv.INTER_CUBIC)
                new_file_path = 'genres/' + \
                    name_genre[artist] + '/' + name
                try:
                    if not cv.imwrite(new_file_path.replace('jpg', 'png'), dst_im):
                        print("bad write:", new_file_path)
                except Exception as e:
                    print(str(e))
                    print("bad write:", new_file_path)
                    exit()
                i += 1

exit()